In [0]:
#1. Creating checkpoint in case if we restart the stram, the log will help keep track of the last moved file and avoid duplicates

checkpoint_path = "/Volumes/healthcare_catalog/raw_data/checkpoints/bronze_healthcare"
landing_zone = "/Volumes/healthcare_catalog/raw_data/landing_zone/"


# 2. Configure Auto Loader (The Stream Reader)
# cloudFiles.schemaLocation allows Spark to 'remember' the JSON structure

bronze_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{checkpoint_path}/schema")
    .option("cloudFiles.inferColumnTypes", "true") 
    .load(landing_zone))

# 3. Write to the Bronze Table
query = (bronze_stream.writeStream
    .format("delta")
    .option("checkpointLocation", f"{checkpoint_path}/data")
    .option("mergeSchema", "true")
    .trigger(availableNow=True) 
    .toTable("healthcare_catalog.raw_data.bronze_healthcare_raw"))

# Wait for the stream to finish its work
query.awaitTermination()
print("Stream finished")
